In [12]:
#code is from https://youtu.be/2kSPbH4jWME by Dataquest, seen feb. 24. 2023 
#here we import liberies and elements form those functions
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

#the queues are threads so the run in the background
#a queue for messages
messages = Queue()

#a queue for recordings
recordings = Queue()

#Here we create a button called "record_button" and determin it as a widgets so we can interacet with it.
#then we set the properties for, such as style, color, when its off and its icon.
record_button = widgets.Button(
    description="Record",
    disabled = False,
    button_style="success",
    icon="microphone"
)

#here we do the same thing, as the one above, but for a stop button insted
stop_button = widgets.Button(
    description="Stop",
    disabled = False,
    button_style="warning",
    icon="stop"
)

#this is the function we run to get a output
output = widgets.Output()

#this is the functions where the program understan what happens when start recording and stop recording 
def start_recording(data):
    
   #this will add a message to teh queue
    messages.put(True)
    
    with output:
        display("starting...")
        
        #here is where we telle the threads that it should reecord the microphone
        record = Thread(traget=record_microphone)
        record.start()
        
        #here we telle the threads to transcribe the audio 
        transcribe = Thread(target=speedh_recognition,args=(output))
        transcribe.start()
    
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
        
#here we tell it what happens when the buttons are pressed
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

#this displays the buttons and the trancript
display(record_button,stop_button,output)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [ ]:
import pyaudio

p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()


In [ ]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    input = True
                    input_device_index=1,
                    frames_per_buffer=chunk)
    frames = []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
        recording.put(frames.copy())
        Frames = []
        
    stream.stop_stream()
    stream.close()
    p.terminate()